# VL05 - Language Modeling with N-grams
In this seminar we explore the concepts behind language modeling and n-grams.

In [ ]:
import spacy
from datasets import load_dataset
import pandas as pd

# load the small English model
nlp = spacy.load("en_core_web_sm")

## 1. Loading the dataset
We will use the a sample of articles from wikipedia. In particular, the `Salesforce/wikitext` dataset from Huggingface. If you don't have access to internet from your notebook, use the script `download_dataset.py` and then `load_from_disk()`

In [ ]:
ds = load_dataset("Salesforce/wikitext", "wikitext-2-raw-v1")
train_texts = ds["train"]["text"]  # list of strings
eval_texts = ds["test"]["text"]

In [ ]:
# Inspect the dataset
df = ds["train"].to_pandas()
df.info()
df["text"].head()

## 2. Building Bi-grams from a Corpus

To estimate a bigram model, we first need to **extract word pairs** (bi-grams) from our text.  
Each bigram represents two consecutive words, (w_{i-1}, w_i), which we’ll later turn into conditional probabilities (P(w_i | w_{i-1})).

**Step 1: Clean the raw text**
The Wikipedia articles in our dataset contain markup such as headings (`=== Section ===`), templates (`{{ ... }}`), or links (`[[link]]`).  
We remove these artifacts to get plain text sentences suitable for tokenization.

**Step 2: Tokenize and collect bigrams**
For each cleaned line:
1. Tokenize it using spaCy (or another tokenizer).  
2. Add `<s>` and `</s>` markers to denote sentence boundaries.  
3. Create consecutive word pairs and count them using Python’s `Counter`.

The resulting counts (e.g., `Counter({("of", "the"): 16949, ("in", "the"): 10560, ...})`)  
form the basis for estimating probabilities and building a **bigram language model**.

In [ ]:
import re
from collections import Counter

def clean_wikitext(lines):
    """Remove obvious wikitext artifacts."""
    for t in lines:
        t = t.strip()
        if not t:
            continue                          # skip blank lines (caused your <s>,</s> spike)
        if re.match(r"^=+\s.*\s=+$", t):
            continue                          # skip section headings like "== History =="
        # Strip templates/links (conservative to avoid nuking content)
        t = re.sub(r"\{\{.*?\}\}", "", t)     # {{ ... }}
        t = re.sub(r"\[\[|\]\]", "", t)       # [[link]] -> link
        yield t

def tokenize(s):
    doc = nlp.make_doc(s) 
    return [token.text for token in doc]

def bigram_counts(texts):
    B = Counter()
    for raw in clean_wikitext(texts):
        toks = tokenize(raw)
        if not toks: continue
            
        toks = ["<s>"] + toks + ["</s>"]
        B.update(zip(toks, toks[1:]))
    return B

# Example with WikiText
B = bigram_counts(train_texts)
B.most_common(50)

## 3. Next token proabilities

To compute the probability of P(w_i | w_{i-1}) we need to compute:
- count(w_{i-1}, w_i) : the total counts of each of the bigrams
- count(w_{i-1}): the total count for the word w_{i-1} as a prefix

We already have the total counts for the bigrams in the counter `B`. We can create another counter called `prefix_counts` to keep track of the counts for the prefix (w_{i-1}).

### 3.1 Prob MLE
We turn counts into probabilities with the Maximul likelihood estimate:
$$P(w_i | w_{i-1}) = count(w_{i-1}, w_i) / count(w_{i-1})$$

In [ ]:
# Given: B = bigram_counts(train_texts) 

# prefix counts: C(w_{i-1}, *)
prefix_counts = Counter()
for (w1, w2), c in B.items():
    prefix_counts[(w1,)] += c


In [ ]:
def prob_mle(next_word, history, B, prefix_counts):
    # history = (w_{i-1},)
    c_bigram = B.get((history[0], next_word), 0)
    c_prefix = prefix_counts.get(history, 0)
    return (c_bigram / c_prefix) if c_prefix else 0.0

In [ ]:
print("C('of','the') =", B.get(('of','the'), 0))
print("C('of', *)   =", prefix_counts.get(('of',), 0))

prob_mle("the", ("of",), B, prefix_counts)

### 3.2 Prob Laplace
One way to avoid 0 probabilities is Laplace smoothing. This is a case of Add-k smoothing, were alpha = 1.

P(w_i | w_{i-1}) = ( count(w_{i-1}, w_i) + alpha) / ( count(w_{i-1}) + alpha * V_size)


In [ ]:
# build vocabulary from observed tokens in the bigrams
def build_vocab_from_bigrams(B):
    V = set()
    for (w1, w2) in B.keys():
        V.add(w1); V.add(w2)
    return V

VOCAB = build_vocab_from_bigrams(B)
V_size = len(VOCAB)
print ("Vocabulary size: ", V_size)

def prob_laplace(next_word, history, B, prefix_counts, V_size, alpha=1):
    c_bigram = B.get((history[0], next_word), 0)
    c_prefix = prefix_counts.get(history, 0)
    return (c_bigram + alpha) / (c_prefix + alpha*V_size)

print("prob_laplace[of the]: ", prob_laplace("the", ("of",), B, prefix_counts, V_size))
print("prob_mle: [of the]", prob_mle("the", ("of",), B, prefix_counts))

print("prob_laplace[of Bielefeld]: ", prob_laplace("Bielefeld", ("of",), B, prefix_counts, V_size))
print("prob_mle: [of Bielefeld]", prob_mle("Bielefeld", ("of",), B, prefix_counts))

### 3.3 Inspect TopK next tokens

In [ ]:
import random
from collections import defaultdict
from functools import partial

# functools.partial lets you pre-fill some arguments of a function and get back a new, simpler function.
#   instead of using lambda functions for prob_fn
#   lambda w,h: prob_mle(w, h, B, prefix_counts)
P_MLE     = partial(prob_mle,     B=B, prefix_counts=prefix_counts)
P_LAPLACE = partial(prob_laplace, B=B, prefix_counts=prefix_counts, V_size=V_size, alpha=1.0)


def topk_next(prev, prob_fn, k=10):
    """
    Show the top-k most likely next words given a previous word.

    prev:      the conditioning word (w_{i-1})
    prob_fn:   a function returning P(next_word | history)
    k:         number of results to display
    """
    candidates = []
    for (w1, w2) in B.keys():
        if w1 == prev:
            p = prob_fn(w2, (prev,))
            if p > 0:
                candidates.append((w2, p))

    # sort by probability (highest first)
    candidates.sort(key=lambda x: x[1], reverse=True)

    return candidates[:k]


print("MLE - Top after 'of':", topk_next("the", P_MLE))
print("LAP - Top after 'of':", topk_next("the", P_LAPLACE))

## 4. Sentence Probabilities

So far, we have probabilities for **individual word pairs** (bigrams).  
To evaluate entire **sentences**, we combine these local probabilities using the **chain rule**:

$$
P(w_1, \ldots, w_n) = \prod_{i=1}^{n} P(w_i \mid w_{i-1})
$$

In log space, this becomes a sum of log-probabilities — much more stable numerically.

### What the functions do
- **`sent_logprob`**  
  Computes the total log-probability of a sentence and returns both:
  - the sum of log probabilities (`lp`)
  - the number of predicted tokens (`N`)

- **`sent_prob`**  
  Converts the log-probability back into the actual (tiny) probability value.  
  Returns `0.0` for sentences containing unseen n-grams.

These functions allow us to compare how **different models** (e.g., MLE vs. Laplace) assign probabilities to entire sentences rather than individual word pairs.

In [ ]:
import math

def sent_logprob(tokens, prob_fn):
    # predict every token except <s>
    toks = ["<s>"] + tokens + ["</s>"]
    N = len(toks) - 1
    if N <= 0:
        return -math.inf, 0
    lp = 0.0
    for i in range(1, len(toks)):
        p = prob_fn(toks[i], (toks[i-1],))
        if p == 0.0:
            return -math.inf, N
        lp += math.log(p)
    return lp, N

def sent_prob(tokens, prob_fn):
    lp, _ = sent_logprob(tokens, prob_fn)
    return 0.0 if lp == -math.inf else math.exp(lp)

In [ ]:
# Example sentence:
test_sentence = "The force will be with me."

# We used spacy earlier - we apply the same tokenisation here
toks = tokenize(test_sentence)

# MLE vs Laplace
p_mle  = sent_prob(toks, P_MLE)
p_lap  = sent_prob(toks, P_LAPLACE)
lp_mle, _ = sent_logprob(toks, P_MLE)
lp_lap, _ = sent_logprob(toks, P_LAPLACE)

print("Sentence prob (MLE):     ", p_mle)
print("Sentence prob (Laplace): ", p_lap)
print("Log-prob (MLE):          ", lp_mle)
print("Log-prob (Laplace):      ", lp_lap)

## 5. Next token prediction

In [ ]:
import random
from collections import defaultdict

# 1) Index successors once: w1 -> list of (w2, count)
successors = defaultdict(list)
for (w1, w2), c in B.items():
    successors[w1].append((w2, c))

# 2) Unigram backoff if a word has no successors
unigram_counts = {h[0]: c for h, c in prefix_counts.items()}

def sample_next(prev):
    """
    Sample next word from the empirical MLE P(. | prev).
    If 'prev' has no observed successors, back off to unigram counts.
    """
    pairs = successors.get(prev)
    if pairs:  # use observed bigram counts as weights
        words, weights = zip(*pairs)
    else:      # back off to unigram counts
        words, weights = zip(*unigram_counts.items())
    return random.choices(words, weights=weights, k=1)[0]

# Example: sample a few next words
random.seed(0)
for h in ["of", "in", "<s>"]:
    print(h, "→", [sample_next(h) for _ in range(5)])

## 6. Sentence generation

Once we can sample the next word $( w_i \sim P(w_i \mid w_{i-1}) )$,  
we can generate full sentences by repeatedly predicting and sampling the next token.

We start from the sentence-begin marker `<s>` and continue until either:
- the model emits the end marker `</s>`, or  
- we reach a maximum length.

This produces sentences that reflect the statistical structure of the corpus —  
frequent word pairs appear naturally, and the text roughly “sounds like” the training data.

In [ ]:
def generate_sentence(max_len=25, start_token="<s>", end_token="</s>"):
    """Generate a random sentence using the bigram model."""
    prev = start_token
    out = []
    for _ in range(max_len):
        nxt = sample_next(prev)
        if nxt == end_token:
            break
        out.append(nxt)
        prev = nxt
    return out

# Try it
random.seed(7)
print(" ".join(generate_sentence()))

## 7. Evaluation

We now evaluate our trained language models using **cross-entropy** and **perplexity** on held-out data.

- **Cross-entropy (H):** measures the average “surprise” of the model on unseen text.  
  Lower values mean better predictions.
- **Perplexity (PP):** exp(H); indicates how many choices the model “considers possible” at each step.  
  Lower PP = less uncertainty.

To ensure fair evaluation:
1. Use the same preprocessing as during training (tokenization, casing, padding).  
2. Map unseen tokens to `<UNK>` so all models face the same vocabulary.  
3. Compare models using the same metric on the same test set.

In [ ]:
import math
from functools import partial

def cross_entropy_perplexity(texts, prob_fn, tokenize_fn):
    """
    Compute corpus cross-entropy (nats/token) and perplexity for a bigram model.

    texts:       iterable of raw strings (sentences or lines)
    prob_fn:     callable(next_word:str, history:tuple[str]) -> float
                 (should return 0.0 for unseen events)
    tokenize_fn: callable(text:str) -> list[str]
    """
    total_logprob = 0.0
    total_tokens  = 0
    skipped_sents = 0

    for text in texts:
        toks = tokenize_fn(text)
        # pad with sentence markers for bigrams: predict every token except <s>
        toks = ["<s>"] + toks + ["</s>"]
        if len(toks) < 2:
            continue

        sent_lp = 0.0
        N = 0
        zero_prob = False

        for i in range(1, len(toks)):
            nxt = toks[i]
            hist = (toks[i-1],)          # bigram history
            p = prob_fn(nxt, hist)
            if p == 0.0:
                zero_prob = True
                break
            sent_lp += math.log(p)
            N += 1

        if zero_prob:
            skipped_sents += 1
            continue

        total_logprob += sent_lp
        total_tokens  += N

    if total_tokens == 0:
        return math.inf, math.inf, skipped_sents

    H  = - total_logprob / total_tokens     # cross-entropy (nats/token)
    PP = math.exp(H)                        # perplexity
    return H, PP, skipped_sents

# define the partial functions for simplicity
P_MLE     = partial(prob_mle,     B=B, prefix_counts=prefix_counts)
P_LAPLACE = partial(prob_laplace, B=B, prefix_counts=prefix_counts, V_size=V_size, alpha=0.1)

clean_eval = list(clean_wikitext(eval_texts))

H_mle, PP_mle, skipped_mle = cross_entropy_perplexity(clean_eval, P_MLE, tokenize)
H_lap, PP_lap, skipped_lap = cross_entropy_perplexity(clean_eval, P_LAPLACE, tokenize)

print(f"MLE      → H: {H_mle:.4f} nats | PP: {PP_mle:.2f} | skipped: {skipped_mle}")
print(f"Lapl(0.1)→ H: {H_lap:.4f} nats | PP: {PP_lap:.2f} | skipped: {skipped_lap}")

## 8. N-Grams with NLTK


### 8.1 Preparing the training dataset

In [ ]:
%%time
from nltk.lm import Lidstone, MLE  # good default; try KneserNeyInterpolated too
from nltk.lm.preprocessing import padded_everygram_pipeline

# 1) Build a minimal pipeline: tokenizer + sentencizer (no tagger/ner/parser)
nlp = spacy.blank("en")
nlp.add_pipe("sentencizer")  # rule-based sentence boundaries

# 2) Batched processing for speed
def to_sents(lines, batch_size=1000):
    sents = []
    for doc in nlp.pipe((t for t in lines if t and t.strip()), batch_size=batch_size):
        for s in doc.sents:
            toks = [tok.text for tok in s if not tok.is_space]
            if toks:
                sents.append(toks)
    return sents

clean_text = clean_wikitext(train_texts) # function defined earlier
sents_train = to_sents(clean_text)

### 8.2 Training the model

In [ ]:
%%time

n = 3
train_ngrams, train_vocab = padded_everygram_pipeline(n, sents_train)
lm = MLE(n)
#lm = Lidstone(0.1, n) # add-k smoothing, alpha=1 : laplace smoothing
lm.fit(train_ngrams, train_vocab)

### 8.3 Inspecting the score 

In [ ]:
lm.score("the", ["end", "of"])   # bigram example

### 8.4 Text Generation with N-gram Models

`lm.generate()` is NLTK’s built-in function for sampling text from a language model.  
It repeatedly predicts the next token based on the model’s probability distribution:

```python
lm.generate(num_words, text_seed=["<s>"])
```

However, this function is **very simple**:
- It samples words **directly from the model’s learned probabilities**.
- It **does not support** parameters like *temperature*, *top-k*.
- You get the model’s “true” random generation — sometimes coherent, sometimes repetitive.

We will see how to address this later.

In [ ]:
lm.generate(20, text_seed=["<s>"])     # 20 tokens

## 9. Evaluating N-gram Language Models

We now evaluate our trained language models using **cross-entropy** and **perplexity** on held-out data.

- **Cross-entropy (H):** measures the average “surprise” of the model on unseen text.  
  Lower values mean better predictions.
- **Perplexity (PP):** exp(H); indicates how many choices the model “considers possible” at each step.  
  Lower PP = less uncertainty.

To ensure fair evaluation:
1. Use the same preprocessing as during training (tokenization, casing, padding).  
2. Map unseen tokens to `<UNK>` so all models face the same vocabulary.  
3. Compare models using the same metric on the same test set.

### 9.1 Training the Models

We train multiple configurations to compare performance:

- **MLE:** maximum likelihood estimate (no smoothing, zeros for unseen n-grams).  
- **Lidstone(0.1):** additive smoothing with α=0.1 (assigns small probability to unseen n-grams).  
- Both **bigrams (n=2)** and **trigrams (n=3)** are trained for comparison.

In [ ]:
%%time

def train_ngrams(sents_train, lm, n=2):
    train_ngrams, train_vocab = padded_everygram_pipeline(n, sents_train)
    lm.fit(train_ngrams, train_vocab)
    return lm

#lm = WittenBellInterpolated(n)           
#lm = KneserNeyInterpolated(n)  

# --- bigrams (n=2) ---
lm2_mle   = train_ngrams(sents_train, MLE(2),            n=2)
lm2_addk  = train_ngrams(sents_train, Lidstone(0.1, 2),  n=2)

# --- trigrams (n=3) ---
lm3_mle   = train_ngrams(sents_train, MLE(3),            n=3)
lm3_addk  = train_ngrams(sents_train, Lidstone(0.1, 3),  n=3)


### 9.2 Defining Evaluation Metrics

We evaluate each model on a test set as before, using:

- **Cross-entropy:** average negative log-probability per token.  
- **Perplexity:** exp(H), lower is better.

To make this fair:
- We pad each sentence with the same `<s>` and `</s>` markers used in training.  
- Unseen tokens are mapped to `<UNK>` to avoid zero probabilities.

We redefine the functions to use the `nltk` functions.

In [ ]:
import math

def sent_cross_entropy(lm, toks):
    import math
    n = lm.order
    # pad like training: (n-1) <s>, and one </s> at the end
    toks = (["<s>"] * (n - 1)) + toks + ["</s>"]
    lp = 0.0
    for i in range(n - 1, len(toks)):
        history = toks[i - (n - 1): i]       # last n-1 tokens
        p = lm.score(toks[i], history)
        if p == 0.0:
            return math.inf
        lp += math.log(p)
    N = len(toks) - (n - 1)                  # predicted tokens
    return -lp / N

def map_oov(tokens, vocab):
    return [w if w in vocab else vocab.unk_label for w in tokens]

def corpus_perplexity(lm, sents):
    Hs = []
    for s in sents:
        toks = map_oov(s, lm.vocab)          # map to <UNK> if needed
        h = sent_cross_entropy(lm, toks)
        Hs.append(h)
        
    finite = [h for h in Hs if math.isfinite(h)]
    if not finite:
        return math.inf, math.inf, len(Hs)
    H = sum(finite) / len(finite)
    PP = math.exp(H)
    skipped = len(Hs) - len(finite)
    return H, PP, skipped

### 9.3 Preparing the Evaluation Data

We clean and tokenize the held-out corpus using the same preprocessing as training.
This ensures that both train and test data follow the same format, vocabulary, and padding.

In [ ]:
clean_text = clean_wikitext(eval_texts) # function defined earlier
sents_eval = to_sents(clean_text)

### 9.4 Comparing and Interpreting Results

We compute cross-entropy, perplexity, and the number of skipped (infinite) sentences for each model.

**How to interpret:**
- Lower **cross-entropy** and **perplexity** → better predictive fit.  
- Many **skipped** sentences → model assigns zero probability to unseen events (typical of MLE).  
- Very high perplexity → model is over-smoothed or poorly calibrated.  
- Trigrams often outperform bigrams (more context), but can overfit without smoothing.

The table below summarizes the results for all configurations.

In [ ]:
from collections import OrderedDict

models = OrderedDict({
    "Bigram MLE":              lm2_mle,
    "Bigram Lidstone(0.1)":    lm2_addk,
    "Trigram MLE":             lm3_mle,
    "Trigram Lidstone(0.1)":   lm3_addk,
    # Add more if you like
    #"Trigram Lidstone (0.05)":    train_ngrams(sents_train, Lidstone(0.05, 3), n=3),    
    # "Fourgram Lidstone (0.05)":    train_ngrams(sents_train, Lidstone(0.05, 4), n=4),
})

rows = []
for name, lm in models.items():
    H, PP, skipped = corpus_perplexity(lm, sents_eval)
    rows.append({
        "Model": name,
        "n": lm.order,
        "Cross-Entropy (nats)": H,
        "Perplexity": PP,
        "Skipped": skipped
    })

df = pd.DataFrame(rows).sort_values("Perplexity", ascending=True)

# Clean display
fmt = {
    "Cross-Entropy (nats)": "{:.4f}".format,
    "Perplexity": "{:.2f}".format
}
display(df.style.format(fmt))


### 9.5 Testing Text Generation

We can compare the generated text after sampling next K words with different ngram models.




In [ ]:
import random, re

def detok(tokens):
    s = " ".join(tokens)
    s = re.sub(r"\s+([.,;:!?])", r"\1", s)
    s = re.sub(r"\s+'\s*s\b", "'s", s)
    return s

def gen_sentence(lm, max_len=25, seed=None, start_token="<s>", end_token="</s>"):
    """Generate a sentence using the model's native sampling (lm.generate)."""
    n = lm.order
    hist = ["<s>"] * (n - 1)
    if seed:
        hist += seed[-(n - 1):]  # use the TAIL of the seed

    out = []
    for _ in range(max_len):
        nxt = lm.generate(1, text_seed=hist[-(n - 1):])
        if nxt == end_token:
            break
        out.append(nxt)
        hist.append(nxt)
    return detok(out)

# Compare models with the same seed & RNG for fairness
random.seed(7)
seed_tokens = ["In", "the"]
for name, lm in models.items():
    print(f"--- {name} ---")
    print(gen_sentence(lm, max_len=25, seed=seed_tokens), "\n")